In [7]:
bad_files = [f for f in path_list if not f.split('.')[0].isdigit()]
print("可疑文件:", bad_files)

# 命令行删除所有
# find . -name ".ipynb_checkpoints" -type d -exec rm -rf {} +

可疑文件: []


In [14]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import time
import math
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.nn.modules.loss import _Loss
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

import warnings
warnings.filterwarnings('ignore')

#创建函数

def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

def compute_psnr(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def compute_mse(img1,img2):
    mse=np.mean( (img1/255. - img2/255.) ** 2 )
    return mse

In [13]:
# Initialize generator 
import HDAMS_Net
from thop import profile

#input = torch.randn(1, 3, 256, 256)
#generator = ULite.ULite().cpu()
#flops, params = profile(generator, inputs=(input, ))
#print('flops:{}'.format(flops))
#print('params:{}'.format(params))

generator = HDAMS_Net.HDAMS_Net().cuda()
generator.load_state_dict(torch.load("./third_pth/LightUNet_30.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./first_pth/LightUNet_170.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./second_pth/LightUNet_170.pth")) ############################ 这里修改模型

# generator.eval() 是告诉 PyTorch 在推理时关闭 Dropout、固定 BatchNorm 的统计方式
generator.eval()  # 模型架构

#########################################################
#########################################################

# 生成Test/output/图片

path= '/root/LU2Net-master/LSUI/Test/test/'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))  # 按照数字排序

i=1
print(len(path_list))
for item in path_list:
    # print(item)  # 打印图片序列
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx).to('cuda')
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx)
    #print(imgx.shape)
    output=generator(imgx)
    out=output.data
    save_image(out, '/root/LU2Net-master/LSUI/Test/output/'+item, nrow=5, normalize=True)
    i=i+1


#########################################################
#########################################################

# # 计算PSNR及SSIM

# path1='/root/LU2Net-master/LSUI/Test/output/'#要改
# path2='/root/LU2Net-master/LSUI/Test/GT/'#要改

# path_list1 = os.listdir(path1)
# path_list1.sort(key=lambda x:int(x.split('.')[0]))

# path_list2 = os.listdir(path2)
# path_list2.sort(key=lambda x:int(x.split('.')[0]))
# PSNR=[]
# SSIM=[]

# from skimage.metrics import structural_similarity as ssim

# for item in path_list1:
#     impath1=path1+item
#     impath2=path2+item
#     imgx= cv2.imread(impath1)
#     imgx=cv2.resize(imgx,(256,256))
    
#     imgy= cv2.imread(impath2)
#     imgy=cv2.resize(imgy,(256,256))
    
#     #print(imgx.shape)
#     psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
#     psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
#     psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])

#     #imgx=imgx.transpose(2,0,1)
#     #imgy=imgy.transpose(2,0,1)
    
#     s_value1 = ssim(imgx,imgy,channel_axis=2)
    

#     psnr=(psnr1+psnr2+psnr3)/3.0
#     Ssim = s_value1
#     PSNR.append(psnr)
#     SSIM.append(Ssim)
# print(np.mean(PSNR))
# print(np.mean(SSIM))


#########################################################
#########################################################

# 计算PSNR、SSIM，并加入 LPIPS

from skimage.metrics import structural_similarity as ssim

# ----------- 新增：LPIPS --------------
import lpips
lpips_alex = lpips.LPIPS(net='alex').cuda()   # 使用 AlexNet 版本（标准）
# --------------------------------------

path1='/root/LU2Net-master/LSUI/Test/output/'   # output
path2='/root/LU2Net-master/LSUI/Test/GT/'       # GT

path_list1 = os.listdir(path1)
path_list1.sort(key=lambda x:int(x.split('.')[0]))

path_list2 = os.listdir(path2)
path_list2.sort(key=lambda x:int(x.split('.')[0]))

PSNR=[]
SSIM=[]
LPIPS_list=[]       # 新增 LPIPS 列表

for item in path_list1:
    impath1=path1+item
    impath2=path2+item
    
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    
    # ---------------- 计算 PSNR ----------------
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
    psnr=(psnr1+psnr2+psnr3)/3.0

    # ---------------- 计算 SSIM ----------------
    ssim_value = ssim(imgx, imgy, channel_axis=2)

    # ---------------- 计算 LPIPS (新增) ----------------
    # 转为 tensor 并归一化到 [-1,1]
    t1 = torch.from_numpy(imgx).permute(2,0,1).unsqueeze(0).float().cuda() / 255.0
    t2 = torch.from_numpy(imgy).permute(2,0,1).unsqueeze(0).float().cuda() / 255.0

    t1 = (t1 - 0.5) * 2       # 转成 LPIPS 标准输入 [-1,1]
    t2 = (t2 - 0.5) * 2

    lpips_value = lpips_alex(t1, t2).item()

    # 保存指标
    PSNR.append(psnr)
    SSIM.append(ssim_value)
    LPIPS_list.append(lpips_value)

print("Average PSNR:", np.mean(PSNR))
print("Average SSIM:", np.mean(SSIM))
print("Average LPIPS:", np.mean(LPIPS_list))    # 新增输出

830
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /root/miniconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Average PSNR: 22.796332830071723
Average SSIM: 0.8801592768404722
Average LPIPS: 0.17611685601702656


In [3]:
# 0
print(np.mean(PSNR))
print(np.mean(SSIM))

20.982803010102273
0.8497829950294534


In [ ]:
# 10
print(np.mean(PSNR))
print(np.mean(SSIM))

In [6]:
# 20
print(np.mean(PSNR))
print(np.mean(SSIM))

22.050164746740073
0.8740461952660525


In [8]:
from thop import profile
for epoch in range(0, 1490, 10):   # 0, 10, 20, ... 90
    # print(f"====== 正在评估 LightUNet_{epoch}.pth ======")
    generator = LU2Net.LU2Net().cuda()
    generator.load_state_dict(torch.load(f"./LightUNet_{epoch}.pth"))  # 这里随 epoch 改
    generator.eval()

    #########################################################
    # 生成Test/output/图片
    path= '/root/LU2Net-master/LSUI/Test/test/'#要改
    path_list = os.listdir(path)
    path_list.sort(key=lambda x:int(x.split('.')[0]))

    i=1
    # print(len(path_list))
    for item in path_list:
        impath=path+item
        imgx= cv2.imread(impath)
        imgx=cv2.resize(imgx,(256,256))
        imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
        imgx = np.array(imgx).astype(dtype)

        imgx= torch.from_numpy(imgx).to('cuda')
        imgx=imgx.permute(2,0,1).unsqueeze(0)
        imgx=imgx/255.0
        imgx = Variable(imgx)

        output=generator(imgx)
        out=output.data
        save_image(out, f'/root/LU2Net-master/LSUI/Test/output/{item}', nrow=5, normalize=True)
        i=i+1

    #########################################################
    # 计算PSNR及SSIM
    path1='/root/LU2Net-master/LSUI/Test/output/'#要改
    path2='/root/LU2Net-master/LSUI/Test/GT/'#要改

    path_list1 = os.listdir(path1)
    path_list1.sort(key=lambda x:int(x.split('.')[0]))
    path_list2 = os.listdir(path2)
    path_list2.sort(key=lambda x:int(x.split('.')[0]))

    PSNR=[]
    SSIM=[]

    for item in path_list1:
        impath1=path1+item
        impath2=path2+item
        imgx= cv2.imread(impath1)
        imgx=cv2.resize(imgx,(256,256))
        imgy= cv2.imread(impath2)
        imgy=cv2.resize(imgy,(256,256))

        psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
        psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
        psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
        psnr=(psnr1+psnr2+psnr3)/3.0

        s_value1 = ssim(imgx,imgy,channel_axis=2)

        PSNR.append(psnr)
        SSIM.append(s_value1)

    print(f"模型 LightUNet_{epoch}.pth → 平均 PSNR: {np.mean(PSNR):.4f}, 平均 SSIM: {np.mean(SSIM):.4f}")

模型 LightUNet_0.pth → 平均 PSNR: 19.5161, 平均 SSIM: 0.8267
模型 LightUNet_10.pth → 平均 PSNR: 21.0201, 平均 SSIM: 0.8682
模型 LightUNet_20.pth → 平均 PSNR: 22.2201, 平均 SSIM: 0.8811
模型 LightUNet_30.pth → 平均 PSNR: 22.7963, 平均 SSIM: 0.8802
模型 LightUNet_40.pth → 平均 PSNR: 22.5067, 平均 SSIM: 0.8818
模型 LightUNet_50.pth → 平均 PSNR: 22.2643, 平均 SSIM: 0.8845
模型 LightUNet_60.pth → 平均 PSNR: 22.8651, 平均 SSIM: 0.8881
模型 LightUNet_70.pth → 平均 PSNR: 22.8364, 平均 SSIM: 0.8863
模型 LightUNet_80.pth → 平均 PSNR: 23.0018, 平均 SSIM: 0.8906
模型 LightUNet_90.pth → 平均 PSNR: 22.8875, 平均 SSIM: 0.8877
模型 LightUNet_100.pth → 平均 PSNR: 22.9731, 平均 SSIM: 0.8890
模型 LightUNet_110.pth → 平均 PSNR: 23.4436, 平均 SSIM: 0.8938
模型 LightUNet_120.pth → 平均 PSNR: 23.2407, 平均 SSIM: 0.8896
模型 LightUNet_130.pth → 平均 PSNR: 23.2920, 平均 SSIM: 0.8900
模型 LightUNet_140.pth → 平均 PSNR: 22.7048, 平均 SSIM: 0.8899
模型 LightUNet_150.pth → 平均 PSNR: 23.8526, 平均 SSIM: 0.8926
模型 LightUNet_160.pth → 平均 PSNR: 22.4520, 平均 SSIM: 0.8834
模型 LightUNet_170.pth → 平均 PSNR: 23.3947, 平

FileNotFoundError: [Errno 2] No such file or directory: './LightUNet_750.pth'

In [14]:
import LU2NetCopy1
from thop import profile
for epoch in range(0, 490, 10):   # 0, 10, 20, ... 90
    # print(f"====== 正在评估 LightUNet_{epoch}.pth ======")
    generator = LU2Net.LU2Net().cuda()
    generator.load_state_dict(torch.load(f"./LightUNet_{epoch}.pth"))  # 这里随 epoch 改
    generator.eval()

    #########################################################
    # 生成Test/output/图片
    path= '/root/LU2Net-master/LSUI/Test/test/'#要改
    path_list = os.listdir(path)
    path_list.sort(key=lambda x:int(x.split('.')[0]))

    i=1
    # print(len(path_list))
    for item in path_list:
        impath=path+item
        imgx= cv2.imread(impath)
        imgx=cv2.resize(imgx,(256,256))
        imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
        imgx = np.array(imgx).astype(dtype)

        imgx= torch.from_numpy(imgx).to('cuda')
        imgx=imgx.permute(2,0,1).unsqueeze(0)
        imgx=imgx/255.0
        imgx = Variable(imgx)

        output=generator(imgx)
        out=output.data
        save_image(out, f'/root/LU2Net-master/LSUI/Test/output/{item}', nrow=5, normalize=True)
        i=i+1

    #########################################################
    # 计算PSNR及SSIM
    path1='/root/LU2Net-master/LSUI/Test/output/'#要改
    path2='/root/LU2Net-master/LSUI/Test/GT/'#要改

    path_list1 = os.listdir(path1)
    path_list1.sort(key=lambda x:int(x.split('.')[0]))
    path_list2 = os.listdir(path2)
    path_list2.sort(key=lambda x:int(x.split('.')[0]))

    PSNR=[]
    SSIM=[]

    for item in path_list1:
        impath1=path1+item
        impath2=path2+item
        imgx= cv2.imread(impath1)
        imgx=cv2.resize(imgx,(256,256))
        imgy= cv2.imread(impath2)
        imgy=cv2.resize(imgy,(256,256))

        psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
        psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
        psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
        psnr=(psnr1+psnr2+psnr3)/3.0

        s_value1 = ssim(imgx,imgy,channel_axis=2)

        PSNR.append(psnr)
        SSIM.append(s_value1)

    print(f"模型 LightUNet_{epoch}.pth → 平均 PSNR: {np.mean(PSNR):.4f}, 平均 SSIM: {np.mean(SSIM):.4f}")

模型 LightUNet_0.pth → 平均 PSNR: 20.9828, 平均 SSIM: 0.8498
模型 LightUNet_10.pth → 平均 PSNR: 22.0208, 平均 SSIM: 0.8774
模型 LightUNet_20.pth → 平均 PSNR: 22.0502, 平均 SSIM: 0.8740
模型 LightUNet_30.pth → 平均 PSNR: 23.1133, 平均 SSIM: 0.8838
模型 LightUNet_40.pth → 平均 PSNR: 22.7743, 平均 SSIM: 0.8848
模型 LightUNet_50.pth → 平均 PSNR: 22.7548, 平均 SSIM: 0.8865
模型 LightUNet_60.pth → 平均 PSNR: 22.6364, 平均 SSIM: 0.8869
模型 LightUNet_70.pth → 平均 PSNR: 23.0450, 平均 SSIM: 0.8858
模型 LightUNet_80.pth → 平均 PSNR: 22.9376, 平均 SSIM: 0.8890
模型 LightUNet_90.pth → 平均 PSNR: 22.7675, 平均 SSIM: 0.8900
模型 LightUNet_100.pth → 平均 PSNR: 23.1282, 平均 SSIM: 0.8886
模型 LightUNet_110.pth → 平均 PSNR: 23.1059, 平均 SSIM: 0.8897
模型 LightUNet_120.pth → 平均 PSNR: 23.4614, 平均 SSIM: 0.8931
模型 LightUNet_130.pth → 平均 PSNR: 23.3242, 平均 SSIM: 0.8936
模型 LightUNet_140.pth → 平均 PSNR: 23.4424, 平均 SSIM: 0.8910
模型 LightUNet_150.pth → 平均 PSNR: 23.9701, 平均 SSIM: 0.8955
模型 LightUNet_160.pth → 平均 PSNR: 23.4801, 平均 SSIM: 0.8943
模型 LightUNet_170.pth → 平均 PSNR: 23.6083, 平

In [5]:
# 计算每一张图片的指标
import os
import csv
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from torchvision import transforms
from torch.autograd import Variable

# ===================== 路径配置 =====================
model_path = r"/root/LU2Net-master/LightUNet_320.pth"

train_input_dir = r"/root/LU2Net-master/LSUI/Train/train/"
train_gt_dir = r"/root/LU2Net-master/LSUI/Train/GT/"

test_input_dir = r"/root/LU2Net-master/LSUI/Test/test/"
test_gt_dir = r"/root/LU2Net-master/LSUI/Test/GT/"

output_dir = r"/root/LU2Net-master/LSUI/Test/output/"
os.makedirs(output_dir, exist_ok=True)

# ===================== 加载模型 =====================
import LU2Net
LightUNet = LU2Net.LU2Net().cuda()
LightUNet.load_state_dict(torch.load(model_path))
LightUNet.eval()
print(f"✅ 已加载模型：{model_path}")

# ===================== 图像预处理 =====================
transform = transforms.Compose([
    transforms.ToTensor()
])

# ===================== 推理函数 =====================
def infer_and_evaluate(input_dir, gt_dir, output_dir, csv_writer):
    all_images = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    for name in tqdm(all_images, desc=f"Processing {os.path.basename(input_dir)}"):
        input_path = os.path.join(input_dir, name)
        gt_path = os.path.join(gt_dir, name)

        if not os.path.exists(gt_path):
            print(f"⚠️ 跳过：找不到GT {gt_path}")
            continue

        # 读取输入图像
        img = Image.open(input_path).convert("RGB")

        # 🔧 自动调整尺寸到最接近的32倍数（防止cat错误）
        w, h = img.size
        new_w = (w // 32) * 32
        new_h = (h // 32) * 32
        img = img.resize((new_w, new_h), Image.BICUBIC)
        
        img_tensor = transform(img).unsqueeze(0).cuda()

        # 推理
        with torch.no_grad():
            output_tensor = LightUNet(img_tensor)
        output_tensor = torch.clamp(output_tensor, 0., 1.)

        # 转回numpy
        pred = output_tensor.squeeze(0).cpu().permute(1, 2, 0).numpy()
        gt = np.array(Image.open(gt_path).convert("RGB"), dtype=np.float32) / 255.0

        # 保存输出图片
        out_img = (pred * 255).astype(np.uint8)
        Image.fromarray(out_img).save(os.path.join(output_dir, name))


        # 对齐尺寸，防止 ValueError
        if gt.shape != pred.shape:
            from skimage.transform import resize
            h, w = gt.shape[:2]
            pred = resize(pred, (h, w), preserve_range=True, anti_aliasing=True)
        
        # 计算指标
        psnr_value = psnr(gt, pred, data_range=1.0)
        ssim_value = ssim(gt, pred, channel_axis=-1, data_range=1.0)

        csv_writer.writerow([input_dir, name, psnr_value, ssim_value])

# ===================== 运行并保存指标 =====================
csv_path = os.path.join(output_dir, "all_image_metrics.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Set", "Image", "PSNR", "SSIM"])

    infer_and_evaluate(train_input_dir, train_gt_dir, output_dir, writer)
    infer_and_evaluate(test_input_dir, test_gt_dir, output_dir, writer)

print(f"✅ 全部图片的 PSNR/SSIM 已保存到: {csv_path}")


✅ 已加载模型：/root/LU2Net-master/LightUNet_320.pth


Processing : 100%|██████████| 830/830 [01:05<00:00, 12.62it/s]

✅ 全部图片的 PSNR/SSIM 已保存到: /root/LU2Net-master/LSUI/Test/output/all_image_metrics.csv


In [10]:
import os
import shutil
import pandas as pd

# ========== 配置（请确认这些路径是你现在使用的） ==========
train_input_dir = r"/root/LU2Net-master/LSUI/Train/train/"
train_gt_dir    = r"/root/LU2Net-master/LSUI/Train/GT/"

test_input_dir  = r"/root/LU2Net-master/LSUI/Test/test/"
test_gt_dir     = r"/root/LU2Net-master/LSUI/Test/GT/"

output_dir      = r"/root/LU2Net-master/LSUI/Test/output/"  # 你的模型输出（pred）所在目录
csv_path        = os.path.join(output_dir, "all_image_metrics.csv")  # 之前生成的指标文件

# 新数据集基础目录（会生成4个子文件夹）
new_base_dir = r"/root/LU2Net-master/LSUI/Reorganized/"
new_train_input = os.path.join(new_base_dir, "Train", "train")
new_train_gt    = os.path.join(new_base_dir, "Train", "GT")
new_test_input  = os.path.join(new_base_dir, "Test", "test")
new_test_gt     = os.path.join(new_base_dir, "Test", "GT")

for d in [new_train_input, new_train_gt, new_test_input, new_test_gt]:
    os.makedirs(d, exist_ok=True)

# ========== 读取指标表 ==========
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"找不到指标文件：{csv_path}，请先运行评估并生成 all_image_metrics.csv")

metrics = pd.read_csv(csv_path)

# 如果 CSV 没有 'Set' 列（老写法），我们把默认来源设置为 test 输入目录（但是你应该有 Set 列）
if 'Set' not in metrics.columns:
    metrics['Set'] = test_input_dir

# 丢弃异常
metrics = metrics.dropna(subset=["Image", "PSNR", "SSIM"]).reset_index(drop=True)

# ========== 计算综合得分（你可以调节权重） ==========
# 统一量级：把 SSIM 放大到 0-100 后与 PSNR 组合
metrics["Score"] = 0.6 * metrics["PSNR"] + 0.4 * (metrics["SSIM"] * 100)

# ========== 排序并取 top20% ==========
metrics = metrics.sort_values(by="Score", ascending=False).reset_index(drop=True)
total = len(metrics)
k = max(1, int(total * 0.2))   # 前 20%，至少 1 张
top_df = metrics.iloc[:k].copy()
rest_df = metrics.iloc[k:].copy()

print(f"Total images = {total}, top 20% = {len(top_df)}, rest = {len(rest_df)}")

# ========== 辅助函数：根据 CSV 的 Set 字段确定原始 input / GT 路径 ==========
def resolve_src_paths(row):
    """
    返回 (src_input_path, src_gt_path)
    依据 row['Set'] 判断其来源是 train 还是 test。
    若无法判断，则优先从 train_input_dir 中寻找，再从 test_input_dir 中寻找。
    """
    set_field = str(row.get("Set", "")).strip()
    name = row["Image"]

    # 规范化路径便于判断
    set_norm = os.path.normpath(set_field).replace("\\", "/").lower()

    # 优先判断包含关系
    if "train" in set_norm and "lsui" in set_norm or os.path.normpath(train_input_dir).lower() in set_norm:
        src_input = os.path.join(train_input_dir, name)
        src_gt    = os.path.join(train_gt_dir, name)
    elif "test" in set_norm and "lsui" in set_norm or os.path.normpath(test_input_dir).lower() in set_norm:
        src_input = os.path.join(test_input_dir, name)
        src_gt    = os.path.join(test_gt_dir, name)
    else:
        # 容错：先尝试 train，再尝试 test
        cand1 = os.path.join(train_input_dir, name)
        cand2 = os.path.join(test_input_dir, name)
        if os.path.exists(cand1):
            src_input = cand1
            src_gt    = os.path.join(train_gt_dir, name)
        else:
            src_input = cand2
            src_gt    = os.path.join(test_gt_dir, name)
    return src_input, src_gt

# ========== 复制函数 ==========
def safe_copy_file(src_path, dst_dir):
    if not os.path.exists(src_path):
        # 如果找不到文件，打印警告并跳过
        print(f"⚠️ 源文件不存在：{src_path}")
        return False
    os.makedirs(dst_dir, exist_ok=True)
    dst_path = os.path.join(dst_dir, os.path.basename(src_path))
    shutil.copy2(src_path, dst_path)
    return True

# ========== 对 top20%（优秀） 复制到 new_test_*，其余复制到 new_train_* ==========
copied_top = 0
copied_rest = 0

# 处理 top20% -> Test
for _, row in top_df.iterrows():
    name = row["Image"]
    src_input, src_gt = resolve_src_paths(row)
    ok1 = safe_copy_file(src_input, new_test_input)
    ok2 = safe_copy_file(src_gt, new_test_gt)
    if ok1 or ok2:
        copied_top += 1

# 处理其余 -> Train
for _, row in rest_df.iterrows():
    name = row["Image"]
    src_input, src_gt = resolve_src_paths(row)
    ok1 = safe_copy_file(src_input, new_train_input)
    ok2 = safe_copy_file(src_gt, new_train_gt)
    if ok1 or ok2:
        copied_rest += 1

print("复制完成：")
print(f"  已复制到新测试集 (input/GT) 的图片数（按行计数）: {copied_top}")
print(f"  已复制到新训练集 (input/GT) 的图片数（按行计数）: {copied_rest}")

# ========== 可选：保存 new split 的清单 ==========
top_df.to_csv(os.path.join(new_base_dir, "reorganized_top20_list.csv"), index=False)
rest_df.to_csv(os.path.join(new_base_dir, "reorganized_rest80_list.csv"), index=False)

print("✅ 重组完成，目录结构如下：")
print(new_base_dir)
print("  Train/")
print("    train/  (输入图)")
print("    GT/     (对应 GT)")
print("  Test/")
print("    test/   (输入图 - 前20%)")
print("    GT/     (对应 GT - 前20%)")

Total images = 4279, top 20% = 855, rest = 3424
复制完成：
  已复制到新测试集 (input/GT) 的图片数（按行计数）: 855
  已复制到新训练集 (input/GT) 的图片数（按行计数）: 3424
✅ 重组完成，目录结构如下：
/root/LU2Net-master/LSUI/Reorganized/
  Train/
    train/  (输入图)
    GT/     (对应 GT)
  Test/
    test/   (输入图 - 前20%)
    GT/     (对应 GT - 前20%)


In [13]:
# Initialize generator 
import LU2Net
from thop import profile

#input = torch.randn(1, 3, 256, 256)
#generator = ULite.ULite().cpu()
#flops, params = profile(generator, inputs=(input, ))
#print('flops:{}'.format(flops))
#print('params:{}'.format(params))

generator = LU2Net.LU2Net().cuda()
generator.load_state_dict(torch.load("./LightUNet_490.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./first_pth/LightUNet_170.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./second_pth/LightUNet_170.pth")) ############################ 这里修改模型

# generator.eval() 是告诉 PyTorch 在推理时关闭 Dropout、固定 BatchNorm 的统计方式
generator.eval()  # 模型架构

#########################################################
#########################################################

# 生成Test/output/图片

path= '/root/LU2Net-master/LSUI/Reorganized/Test/test/'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))  # 按照数字排序

i=1
print(len(path_list))
for item in path_list:
    # print(item)  # 打印图片序列
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx).to('cuda')
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx)
    #print(imgx.shape)
    output=generator(imgx)
    out=output.data
    save_image(out, '/root/LU2Net-master/LSUI/Reorganized/Test/output/'+item, nrow=5, normalize=True)
    i=i+1


#########################################################
#########################################################

# 计算PSNR及SSIM

path1='/root/LU2Net-master/LSUI/Reorganized/Test/output/'#要改
path2='/root/LU2Net-master/LSUI/Reorganized/Test/GT/'#要改

path_list1 = os.listdir(path1)
path_list1.sort(key=lambda x:int(x.split('.')[0]))

path_list2 = os.listdir(path2)
path_list2.sort(key=lambda x:int(x.split('.')[0]))
PSNR=[]
SSIM=[]

from skimage.metrics import structural_similarity as ssim

for item in path_list1:
    impath1=path1+item
    impath2=path2+item
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    
    #print(imgx.shape)
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])

    #imgx=imgx.transpose(2,0,1)
    #imgy=imgy.transpose(2,0,1)
    
    s_value1 = ssim(imgx,imgy,channel_axis=2)
    

    psnr=(psnr1+psnr2+psnr3)/3.0
    Ssim = s_value1
    PSNR.append(psnr)
    SSIM.append(Ssim)
print(np.mean(PSNR))
print(np.mean(SSIM))

855
25.588941569292174
0.8968015331660121


# 补充

In [16]:
# Initialize generator 
import HDAMS_Net
from thop import profile

#input = torch.randn(1, 3, 256, 256)
#generator = ULite.ULite().cpu()
#flops, params = profile(generator, inputs=(input, ))
#print('flops:{}'.format(flops))
#print('params:{}'.format(params))

generator = HDAMS_Net.HDAMS_Net().cuda()
generator.load_state_dict(torch.load("./third_pth/LightUNet_490.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./first_pth/LightUNet_170.pth")) ############################ 这里修改模型
# generator.load_state_dict(torch.load("./second_pth/LightUNet_170.pth")) ############################ 这里修改模型

# generator.eval() 是告诉 PyTorch 在推理时关闭 Dropout、固定 BatchNorm 的统计方式
generator.eval()  # 模型架构

#########################################################
#########################################################

# 生成Test/output/图片

path= '/root/LU2Net-master/LSUI/Reorganized/Test/test/'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))  # 按照数字排序

i=1
print(len(path_list))
for item in path_list:
    # print(item)  # 打印图片序列
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx).to('cuda')
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx)
    #print(imgx.shape)
    output=generator(imgx)
    out=output.data
    save_image(out, '/root/LU2Net-master/LSUI/Reorganized/Test/output/'+item, nrow=5, normalize=True)
    i=i+1


#########################################################
#########################################################

# 计算PSNR、SSIM，并加入 LPIPS

from skimage.metrics import structural_similarity as ssim

# ----------- 新增：LPIPS --------------
import lpips
lpips_alex = lpips.LPIPS(net='alex').cuda()   # 使用 AlexNet 版本（标准）
# --------------------------------------

path1='/root/LU2Net-master/LSUI/Reorganized/Test/output/'#要改
path2='/root/LU2Net-master/LSUI/Reorganized/Test/GT/'#要改

path_list1 = os.listdir(path1)
path_list1.sort(key=lambda x:int(x.split('.')[0]))

path_list2 = os.listdir(path2)
path_list2.sort(key=lambda x:int(x.split('.')[0]))

PSNR=[]
SSIM=[]
LPIPS_list=[]       # 新增 LPIPS 列表

for item in path_list1:
    impath1=path1+item
    impath2=path2+item
    
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    
    # ---------------- 计算 PSNR ----------------
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
    psnr=(psnr1+psnr2+psnr3)/3.0

    # ---------------- 计算 SSIM ----------------
    ssim_value = ssim(imgx, imgy, channel_axis=2)

    # ---------------- 计算 LPIPS (新增) ----------------
    # 转为 tensor 并归一化到 [-1,1]
    t1 = torch.from_numpy(imgx).permute(2,0,1).unsqueeze(0).float().cuda() / 255.0
    t2 = torch.from_numpy(imgy).permute(2,0,1).unsqueeze(0).float().cuda() / 255.0

    t1 = (t1 - 0.5) * 2       # 转成 LPIPS 标准输入 [-1,1]
    t2 = (t2 - 0.5) * 2

    lpips_value = lpips_alex(t1, t2).item()

    # 保存指标
    PSNR.append(psnr)
    SSIM.append(ssim_value)
    LPIPS_list.append(lpips_value)

print("Average PSNR:", np.mean(PSNR))
print("Average SSIM:", np.mean(SSIM))
print("Average LPIPS:", np.mean(LPIPS_list))    # 新增输出

855
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /root/miniconda3/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth
Average PSNR: 25.588941569292174
Average SSIM: 0.8968015331660121
Average LPIPS: 0.08466249479100718


In [12]:
# 170
print(np.mean(PSNR))
print(np.mean(SSIM))

26.43310373329989
0.9115993233892918


In [ ]:
# 测试模型参数量及算力需求

import torch
from thop import profile

# 模型定义
model = LU2Net.LU2Net().cuda()

# 统计参数量
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Params: {num_params / 1e6:.2f}M")

# 假设输入是 3x256x256 图像，根据你的任务改成实际输入大小
dummy_input = torch.randn(1, 3, 256, 256).to(next(model.parameters()).device)

# 统计 FLOPs 和 Params
flops, params = profile(model, inputs=(dummy_input,))
print(f"FLOPs: {flops / 1e9:.2f}G")
print(f"Params (from thop): {params / 1e6:.2f}M")